### Part 1.4 - Extract_data_20000_movies

#### Import the libraries

In [ ]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook
# Example of Use: tqdm_notebook(examples, desc="Converting examples to features")

#### Fourth 5000 movies (20,000)

Extract data about:

* field 1: Plot Summary
* field 2: Actors
* field 3: Directors
* field 4: IMDB rating
* field 5: Plot Synopsis
* field 6: Reviews

In [ ]:
"""
Open the movie_content urls for the fourth 5000 movies and create the souplist of those URLs
"""
content_four = []

with open(os.path.join(os.getcwd(), 'movie_content_url\\data_four_12012020.pkl'), 'rb') as f:
    
    content_four = pickle.load(f)

print("Number of URLs: {}".format(len(content_four)))

content_souplist_four = []

for i in tqdm_notebook(content_four):
    
    content_souplist_four.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_four)))

In [ ]:
"""
Open the synopsis urls of the fourth 5000 movies and create the souplist of those URLs
"""
synopsis_four = []

with open(os.path.join(os.getcwd(), 'synopsis_url\\synopsis_four_12012020.pkl'), 'rb') as f:
    
    synopsis_four = pickle.load(f)

print("Number of URLs: {}".format(len(synopsis_four)))

synopsis_souplist_four = []

for i in tqdm_notebook(synopsis_four):
    
    synopsis_souplist_four.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(synopsis_souplist_four)))

In [ ]:
"""
Open the reviews urls of the fourth 5000 movies and create the souplist of those URLs
"""
review_four = []

with open(os.path.join(os.getced(), 'reviews_url\\review_four_15012020.pkl'), 'rb') as f:
    
    review_four = pickle.load(f)

print("Number of URLs: {}".format(len(review_four)))

review_souplist_four = []

for i in tqdm_notebook(review_four):
    
    review_souplist_four.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_four)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_four)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_four)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_four)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name if len(item)!=0]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_four)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract plot synopsis

Having extracted the synopsis content for each movie, we noticed that roughly more than 50% percent of the movies have a synopsis text.
Thus, in order not to erase almost half of the first 5,000 we will not further proceed in keeping the synopsis text
"""
synopsis_step_one = []
synopsis_step_two = []
synopsis_step_three = []

[synopsis_step_one.append(i.find_all('ul', {'class':'ipl-zebra-list', 'id':'plot-synopsis-content'})) for i in tqdm_notebook(synopsis_souplist_four)]

[[synopsis_step_two.append(j.find_all('li', {'class':'ipl-zebra-list__item'})) for j in i] for i in synopsis_step_one]

[[synopsis_step_three.append(j.text.strip(' ').replace('\n', '').replace('\\', '')) for j in i] for i in synopsis_step_two]

index_to_remove_no_synopsis = [i for i,x in enumerate(synopsis_step_one) if not x]

print("Length of Synopsis list: {}".format(len(synopsis_step_three)))
print("Length of the list with Movies that don't have synopsis: {}".format(len(index_to_remove_no_synopsis)))
if len(index_to_remove_no_synopsis) == 0:
    print("None of the movies miss a synopsis")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_synopsis))

In [ ]:
"""
Print the movies with no synopsis text.
Result: We observed that many movies missed a synopsis text. Thus, we decided to not use synopsis for furhter analysis (although will be extracted)
"""
print([i for i,x in enumerate(synopsis_step_one) if not x])
print([i for i,x in enumerate(synopsis_step_two) if not x])
print([i for i,x in enumerate(synopsis_step_three) if not x])

# Many of the movies do not have a synopsis text, this is it won't be used for furhter analysis (althpugh will be extracted)

In [ ]:
"""
Field 6: Extract movie reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_four)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
"""
Print the movies with no user reviews text.
"""
print([i for i,x in enumerate(myfield_review_step_one) if not x])
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

<b> - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - </b>

#### Dataframe creation based on the movie content, synopsis and reviews extracted

In [ ]:
"""
Import the movie_content (the extracted HTML document of the column imdb_url)
Import the reviews content (the extracted HTML document of the column reviews_url)
"""
with open(os.path.join(os.getcwd(), 'movie_content_url\\data_four_12012020.pkl'), 'rb') as f:
    
    content_four = pickle.load(f)

print("Number of URLs: {}".format(len(content_four)))

with open(os.path.join(os.getcwd(), 'reviews_url\\review_four_15012020.pkl'), 'rb') as f:
    
    review_four = pickle.load(f)

print("Number of URLs: {}".format(len(review_four)))

In [ ]:
"""
Below we import the 5 fields extracted previously
However, we we imported the "rating" file, we noticed that they did not downloaded properly. Actually, the respective list was empty,
thus we decided to download again the data in order no to make any other mistake with the index of the movies.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\pre-indexed files\\plot_four_17012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\pre-indexed files\\rating_four_17012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\pre-indexed files\\actors_four_17012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\pre-indexed files\\director_four_17012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\pre-indexed files\\synopsis_four_18012020.pkl'), 'rb') as f:
    
    synopsis = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\pre-indexed files\\reviews_four_18012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
print(len(plot))
print(len(rating))
print(len(actors))
print(len(director))
print(len(synopsis))
print(len(reviews))

print(rating) #empty

In [ ]:
"""
Running the commands below we can spot the number of indexes that should be removed from the data extracted.
"Add a Plot": Denoted that the movies does have a written plot on its IMDB website.
"It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.": Also denotes a non-written synopsis.

Synopsis indexes are not further considered. There are too many movies with no synopsis text and there is no poin in erasing almost half of the first 5000 movies due to missing synopsis text.
"""
matching_add_plot = [s for s, x in enumerate(plot) if "Add a Plot" in x]

print("Number of movies with 'Add Plot': ", len(matching_add_plot), '\n')

matching_add_synopsis = [s for s, x in enumerate(synopsis) if 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.' in x]

print("Number of movies with no synopsis text: ", len(matching_add_synopsis))

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[35,110,170,290,365,366,449,573,642,643,714,866,929,1011,1170,1257,1474,1488,1639,1812,1902,1994,2036,2046,2180,2189,2259,2343,2357,2422,2438,2494,2503,2507,2584,2661,2857,2867,3040,3117,3124,3184,3241,3326,3368,3506,3688,3735,3763,3766,3772,3869,3928,3947,4004,4008,4020,4034,4049,4119,4153,4179,4197,4287,4301,4411,4456,4508,4526,4537,4541,4609,4649,4659,4714,4727,4774,4795,4908,4948,4955,152,261,316,343,381,391,529,778,793,876,938,1031,1097,1148,1153,1170,1238,1300,1381,1482,1497,1568,1639,1902,1910,2046,2086,2189,2190,2224,2234,2235,2426,2439,2502,2504,2578,2671,2715,2730,2749,2880,2976,2985,2996,3061,3073,3138,3271,3283,3364,3371,3538,3613,3653,3688,3689,3742,3759,3774,3777,3809,3831,3841,3951,4049,4086,4106,4171,4208,4322,4327,4381,4433,4457,4465,4618,4672,1482,1902,2224,4180,4672,1482,1902,2224,4180,4672,2224,4170,4464,45,146,150,153,160,170,236,290,307,327,343,365,366,369,410,520,539,545,570,575,643,705,714,792,864,866,876,881,889,1086,1101,1142,1148,1151,1173,1202,1254,1257,1286,1412,1414,1482,1488,1522,1639,1664,1738,1759,1813,1856,1902,1933,1934,1950,1989,1994,1995,2023,2028,2036,2046,2086,2189,2193,2259,2268,2321,2344,2355,2368,2414,2422,2438,2439,2442,2443,2486,2494,2507,2508,2517,2518,2578,2584,2596,2605,2670,2786,2810,2852,2913,2975,2995,3071,3084,3117,3119,3124,3130,3146,3173,3174,3184,3229,3238,3281,3282,3290,3368,3369,3466,3489,3506,3518,3519,3535,3537,3538,3576,3610,3631,3666,3688,3695,3699,3715,3759,3762,3767,3774,3789,3809,3830,3887,3899,3902,3910,3928,3936,3947,3977,3979,3983,4008,4020,4049,4066,4119,4137,4159,4167,4170,4179,4184,4185,4186,4216,4222,4231,4250,4290,4338,4362,4363,4364,4376,4399,4405,4411,4464,4508,4515,4518,4540,4541,4546,4573,4576,4584,4587,4588,4594,4649,4659,4666,4671,4675,4695,4716,4727,4736,4775,4793,4862,4872,4881,4892,4910,4919,4937,4947]

index_remove = list(dict.fromkeys(index_remove))

print(len(index_remove))

content_index_four = [i for j, i in enumerate(content_four) if j not in index_remove]

review_index_four = [i for j, i in enumerate(review_four) if j not in index_remove]

print(len(content_index_four))

print(len(review_index_four))

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_four\\content_index_four_20012020.pkl'), 'wb') as f: 
#     pickle.dump(content_index_four, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_four\\review_index_four_20012020.pkl'), 'wb') as f:
#     pickle.dump(review_index_four, f)

In [ ]:
"""
Import the pruned list of movie_content & reviews web pages. The indexes that have been removed did not contain one or more -
of the following fields: plot, actors, director(s), imdb_rating, review(s)

It is important to re-run the process of extracting the information in order to assure that we have extracted the correct information and text for each movie
Otherwise, we might observe mismatches in the fields assigned to each movie title.
"""
content_souplist_four = []

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\content_index_four_20012020.pkl'), 'rb') as f:
    
    content_index_four = pickle.load(f)

print("Number of URLs: {}".format(len(content_index_four)))

for i in tqdm_notebook(content_index_four):
    
    content_souplist_four.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_four)))

#-----------------------------------------------------------------------------------------------------

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\review_index_four_20012020.pkl'), 'rb') as f:
    
    review_four = pickle.load(f)

print("Number of URLs: {}".format(len(review_four)))

review_souplist_four = []

for i in tqdm_notebook(review_four):
    
    review_souplist_four.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_four)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_four)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_four)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_four)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_four)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract movie user reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_four)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
print(len([i for i,x in enumerate(myfield_review_step_one) if not x]))
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# # Pickle the requests file for further use!

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\plot_four_20012020.pkl'), 'wb') as f:
#     pickle.dump(plot_summary, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\actors_four_20012020.pkl'), 'wb') as f:
#     pickle.dump(actors_list, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\director_four_20012020.pkl'), 'wb') as f:
#     pickle.dump(director_names, f)

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\reviews_four_24012020.pkl'), 'wb') as f:
#     pickle.dump(myfield_review_step_four, f)

In [ ]:
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\rating_four_20012020.pkl'), 'wb') as f:
#     pickle.dump(ratings, f)

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[35,110,170,290,365,366,449,573,642,643,714,866,929,1011,1170,1257,1474,1488,1639,1812,1902,1994,2036,2046,2180,2189,2259,2343,2357,2422,2438,2494,2503,2507,2584,2661,2857,2867,3040,3117,3124,3184,3241,3326,3368,3506,3688,3735,3763,3766,3772,3869,3928,3947,4004,4008,4020,4034,4049,4119,4153,4179,4197,4287,4301,4411,4456,4508,4526,4537,4541,4609,4649,4659,4714,4727,4774,4795,4908,4948,4955,152,261,316,343,381,391,529,778,793,876,938,1031,1097,1148,1153,1170,1238,1300,1381,1482,1497,1568,1639,1902,1910,2046,2086,2189,2190,2224,2234,2235,2426,2439,2502,2504,2578,2671,2715,2730,2749,2880,2976,2985,2996,3061,3073,3138,3271,3283,3364,3371,3538,3613,3653,3688,3689,3742,3759,3774,3777,3809,3831,3841,3951,4049,4086,4106,4171,4208,4322,4327,4381,4433,4457,4465,4618,4672,1482,1902,2224,4180,4672,1482,1902,2224,4180,4672,2224,4170,4464,45,146,150,153,160,170,236,290,307,327,343,365,366,369,410,520,539,545,570,575,643,705,714,792,864,866,876,881,889,1086,1101,1142,1148,1151,1173,1202,1254,1257,1286,1412,1414,1482,1488,1522,1639,1664,1738,1759,1813,1856,1902,1933,1934,1950,1989,1994,1995,2023,2028,2036,2046,2086,2189,2193,2259,2268,2321,2344,2355,2368,2414,2422,2438,2439,2442,2443,2486,2494,2507,2508,2517,2518,2578,2584,2596,2605,2670,2786,2810,2852,2913,2975,2995,3071,3084,3117,3119,3124,3130,3146,3173,3174,3184,3229,3238,3281,3282,3290,3368,3369,3466,3489,3506,3518,3519,3535,3537,3538,3576,3610,3631,3666,3688,3695,3699,3715,3759,3762,3767,3774,3789,3809,3830,3887,3899,3902,3910,3928,3936,3947,3977,3979,3983,4008,4020,4049,4066,4119,4137,4159,4167,4170,4179,4184,4185,4186,4216,4222,4231,4250,4290,4338,4362,4363,4364,4376,4399,4405,4411,4464,4508,4515,4518,4540,4541,4546,4573,4576,4584,4587,4588,4594,4649,4659,4666,4671,4675,4695,4716,4727,4736,4775,4793,4862,4872,4881,4892,4910,4919,4937,4947]
index_remove = list(dict.fromkeys(index_remove))
dataset = pd.read_pickle(os.path.join(os.getcwd(), "dataset_58,000_14012020_latest_version.pkl"))
dataset_four = dataset.iloc[15000:20000].reset_index(drop=True)
dataset_four = dataset_four[~dataset_four.index.isin(index_remove)]
dataset_four.shape

In [ ]:
"""
Loading the data extracted.
Those are the most important data source across all the notebook. 
The whole data extraction part was executed to end up with the following 6 files per mini-batch.

The examiner can directly import the files below to review the movie content extracted for the fourth mini-batch.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\plot_four_20012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\rating_four_20012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\actors_four_20012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\director_four_20012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
        
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_four\\reviews_four_24012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
assert len(plot) == len(rating) == len(actors) == len(director) == len(reviews)

In [ ]:
dataset_four['actors'] = actors
dataset_four['plot'] = plot
dataset_four['imdb_rating'] = rating
dataset_four['director'] = director
dataset_four['reviews'] = reviews
dataset_four = dataset_four.drop(["movieId", "imdbId", "synopsis_url"], axis=1)

In [ ]:
index_remove=[]
index_remove=[2162,2170,2222,2243,2253,2264,2309,2316,2330,2332,2374,2381,2390,2398,2457,2462,2473,2481,2655,2678,2719,2776,2926,2937,2969,2970,2974,2979,2993,3019,3028,3072,3080,3126,3201,3296,3318,3334,3345,3360,3361,3398,3431,3450,3483,3509,3512,3527,3568,3572,3577,3590,3609,3682,3693,3695,3702,3719,3726,3736,3764,3765,3768,3791,3802,3829,3845,3895,3912,3932,3939,3951,3978,3983,3991,4009,4047,4091,4114,4125,4146,4151,4156,4247,4253,4255,4274,4278,4304,4306,4313,4315,4320,4372,4381,4387,4393,4412,4431,4441,4449,4486,4503,4570,4579,4587,4597,4613,4621,4638]
index_remove = list(dict.fromkeys(index_remove))
dataset_four = dataset_four[~dataset_four.index.isin(index_remove)]

In [ ]:
"""
This is the table to use in Part 1, and the table that will be concatenated with the rest of the mini-batch tables.
"""
# dataset_four.to_pickle(os.path.join(os.getcwd(), "58,000 movies\\movie_four\\dataset_four_final_25012020.pkl"))